# The Tensor
#### Previously, we made a distinction between vectors (one-dimensional arrays) and matrices (two-dimensional arrays). When we start working with more complicated neural networks, we'll need to use a higher-dimensional arrays as well.

#### In many neural network libraries, *n*-dimensional arrays are referred to as *tensors*, which is what we'll call them too.
#### If I were writing an entire book about deep learning, I'd implement a full-featured `Tensor` class that overloaded Python's arithmetic operators and could handle a variety of other operations. Such an implementation would take a notebook on its own. Here we'll cheat and say that a `Tensor` is just a `list`. This is true in one direction - all our vectors and matrices and higher-dimensional analogues *are* lists. It is certainly not true in the other direction - most Python *lists* are not *n*-dimensional arrays in our sense

#### First, let's write a helper function to find a tensor's *shape*

In [ ]:
from typing import List

Tensor = list
def shape(tensor: Tensor) -> List[int]:
    sizes: List[int] = []
    while isinstance(tensor, list):
        sizes.append(len(tensor))
        tensor = tensor[0]
    return sizes

print(shape([1, 2, 3]))
print(shape([[1,2], [3,4], [5,6]]))

#### Because tensors can have any number of dimensions, we'll typically need to work with them recursively. We'll do one thing in the one-dimensional case and recurse in the higher-dimensional case:

In [ ]:
def is_1d(tensor: Tensor) -> bool:
    """
    If tensor[0] is a list, it's a higher-order tensor. Otherwise, tensor is 1-dimensional (that is, a vector)
    """
    return not isinstance(tensor[0], list)

print(is_1d([1, 2, 3]))

#### Which we can use to write a recursive `tensor_sum` function:

In [ ]:
def tensor_sum(tensor: Tensor) -> float:
    """ Sums up all the values in the tensor"""
    if is_1d(tensor):
        return sum(tensor)
    else:
        return sum(tensor_sum(tensor_i) for tensor_i in tensor)

print(tensor_sum([1, 2, 3]))

#### We'll create a couple of helper functions so that we don't have to rewrite this logic everywhere. The first applies a function elementwise to a single tensor

In [ ]:
from typing import Callable

def tensor_apply(f: Callable[[float], float], tensor: Tensor) -> Tensor:
    """ Applies f elementwise""" 
    if is_1d(tensor):
        return[f(x) for x in tensor]
    else:
        return [tensor_apply(f, tensor_i) for tensor_i in tensor]

print(tensor_apply(lambda x: x + 1, [1, 2, 3])) # So in this example, we are adding 1 to every instance of x - each tensor

#### We can use this to write a function that creates a zero tensor with the same shape as a given tensor:

In [ ]:
def zeros_like(tensor: Tensor) -> Tensor:
    return tensor_apply(lambda _: 0.0, tensor)

print(zeros_like([1, 2, 3]))

#### We'll also need to apply a function to corresponding elements from two tensors (which had better be the exact same shape, although we won't check that)

In [ ]:
def tensor_combine(f: Callable[[float, float], float],
                    t1: Tensor,
                    t2: Tensor) -> Tensor:
    """ Applies f to corresponding elements of t1 and t2"""
    if is_1d(t1):
        return [f(x, y) for x,y in zip(t1, t2)]
    else:
        return [tensor_combine(f, t1_i, t2_i) for t1_i, t2_i in zip(t1, t2)]

import operator
print(tensor_combine(operator.add, [1, 2, 3], [4, 5, 6]))
print(tensor_combine(operator.mul, [1, 2, 3], [4, 5, 6]))

# The Layer Abstraction 
#### In our previous notebook we built a simple neural net that allowed us to stack two layers of neurons, each of which computed `sigmoid(dot(weights, inputs))`.

#### Although that's perhaps an idealized representation of what an actual neuron does, in practice we'd like to allow a wider variety of things. Perhaps we'd like the neurons to remember something about their previous inputs. Perhaps we'd like to use a different activation function than `sigmoid`. And frequently we'd like to use more than two layers. (Our `feed_forward` function actually handled any number of layers, but our gradient computations did not.)

#### In this notebook we'll build machinery for implementing such a variety of neural networks. Our fundamental abstraction will be the `Layer`, something that knows how to apply some function to its inputs that knows how to backpropagate gradients.

#### One way of thinking about the neural networks we built in `fizzbuzz.ipynb` is as a "linear" layer, followed by a "sigmoid" layer, then another linear layer and another sigmoid layer. We didn't distinguish them in these terms, but doing so will allow us to experiment with much more general structures:

In [ ]:
from typing import Iterable, Tuple

class Layer:
    """
    Our neural networks will be composed of Layers, each of which 
    knows how to do some computation on its inputs in the "forward" 
    direction and propagate gradients in the "backward" direction
    """
    def forward(self, input):
        """
        Not the lack of typyes. We're not going to be prescriptive
        about what kinds of inputs layer can take and what kinds of 
        outputs they can return.
        """     
        raise NotImplementedError

    def backward(self, gradient):
        """
        Similarly, we're not going to be prescriptive about what the
        gradient looks like. It's up to you the user to make sure 
        that you're doing things sensibly.
        """
        raise NotImplementedError
    
    def params(self) -> Iterable[Tensor]:
        """ 
        Returns the parameters of this layer. The default implementation
        return nothing, so that if you have a layer with no parameters
        you don't have to implement this.
        """
        return ()
    
    def grads(self) -> Iterable[Tensor]:
        """
        Returns the gradients, in the same order as params()
        """
        return()

#### The `forward` and `backward` methods will have to be implemented in our concrete subclasses. Once we build a neural net, we'll want to train it using gradient descent, which means we'll want to update each parameter in the network using its gradient. Accordingly, we'll insist that each layer be able to tell us its parameters and gradients

#### Some layers (for example, a layer that applies `sigmoid` to each of its inputs) have no parameters to update, so we provide a default implementation that handles that case.

In [ ]:
import math

def sigmoid(t: float) -> float:
    return 1 / (1 + math.exp(-t))

In [ ]:
class Sigmoid(Layer):
    def forward(self, input: Tensor) -> Tensor:
        """
        Apply sigmoid to each element of the input tensor,
        and save the results to use in backpropagation
        """
        self.sigmoids = tensor_apply(sigmoid, input)
        return self.sigmoids
        
    def backward(self, gradient: Tensor) -> Tensor:
        return tensor_combine(lambda sig, grad: sig * (1 - sig) * grad,
                              self.sigmoids,
                              gradient)

#### It turns out that the initial parameter values can make a huge difference in how quickly (and sometimes *whether*) the network trains. If weights are too big, they may produce large outputs in a range where the activation function has near-zero gradients. And parts of the network that have zero gradients can't learn anything via gradient descent.


#### Accordingly, we'll implement three different schemes for randomly generating our weight tensors. The first is to choose each value from the random uniform distribution on [0, 1] - that is, as a `random.random()`. The second (and default) is to choose each value randomly from a standard normal distribution. And the third is to use *Xavier initialization*, where each weight is initialized with a random draw from a normal distribution with mean 0 and variance 2 / (`num_inputs + num_outputs`). It turns out this often works nicely for neural network weights. We'll implement these with a `random_uniform` function and a `random_normal` function.

In [ ]:
import random
from ml.probability import inverse_normal_cdf

def random_uniform(*dims: int) ->  Tensor:
    if len(dims) == 1:
        return [random.random() for _ in range(dims[0])]
    else:
        return [random_uniform(*dims[1:]) for _ in range(dims[0])]

def random_normal(*dims: int,
                    mean: float = 0.0,
                    variance: float = 1.0) -> Tensor:
    if len(dims) == 1:
        return [mean + variance * inverse_normal_cdf(random.random()) for _ in range(dims[0])]
    else:
        return [random_normal(*dims[1:], mean=mean, variance=variance) for _ in range(dims[0])]

assert shape(random_uniform(2, 3, 4)) == [2, 3, 4]
assert shape(random_normal(5, 6, mean=10)) == [5, 6]

In [ ]:
def random_tensor(*dims: int, init: str = 'normal') -> Tensor:
    if init == 'normal':
        return random_normal(*dims)
    elif init == 'uniform':
        return random_uniform(*dims)
    elif init == 'xavier':
        variance = len(dims) / sum(dims)
        return random_normal(*dims, variance=variance)
    else:
        raise ValueError(f"unknown init: {init}")

#### Now we can define our linear layer. We need to intialize it with the dimension of the inputs (which tells us how many weights each neuron needs), the dimension of the outputs (which tells us how many weights each neuron needs), and the initialization scheme we want:

In [ ]:
from ml.algebra import dot

class Linear(Layer):
    def __init__(self,
                input_dim: int,
                output_dim: int,
                init: str = 'xavier') -> None:
        """
        A layer of output_dim neurons, each with input_dim weights
        (and a bias)
        """
        self.input_dim = input_dim
        self.output_dim = output_dim

        # self.w[o] is the weights for the oth neuron
        self.w = random_tensor(output_dim, input_dim, init=init)

        # self.b[o] is the bias term for the oth neuron
        self.b = random_tensor(output_dim, init=init)

#### The `forward` method is easy to implement. We'll get one output per neuron, which we stick in a vector. And each neuron's output is just the `dot` of its weights with the input, plus its bias:

In [ ]:
    def forward(self, input: Tensor) -> Tensor:
        # Save the input to use in the backward pass.
        self.input = input

        # Return the vector of neuron outputs.
        return [dot(input, self.w[o]) + self.b[o]
                for o in range(self.output_dim)]

In [ ]:
    def backward(self, gradient: Tensor) -> Tensor:
        # Each b[o] gets added to output[o], which means
        # the gradient of b is the same as the output gradient.
        self.b_grad = gradient

        # Each w[o][i] multiplies input[i] and gets added to output[o].
        # So its gradient is input[i] * gradient[o].
        self.w_grad = [[self.input[i] * gradient[o]
                        for i in range(self.input_dim)]
                        for o in range(self.output_dim)]

        # Each input[i] multiplies every w[o][i] and gets added to every
        # output[o]. So its gradient is the sum of w[o][i] * gradient[o]
        # across all the outputs.
        return [sum(self.w[o][i] * gradient[o] for o in range(self.output_dim))
                for i in range(self.input_dim)]

In [ ]:
def params(self) -> Iterable[Tensor]:
    return [self.w, self.b]

def grads(self) -> Iterable[Tensor]:
    return [self.w_grad, self.b_grad]

# Neural Networks as a sequence of layers
#### We'd like to think of neural networks as a sequence of layers, so let's come up with a way to combine multiple layers into one. The resulting neural network is itself a layer, and it implements the `Layer` methods in the obvious ways:

In [ ]:
from typing import List

class Sequential(Layer):
    """
    A layer consisting of a sequence of other layers.
    It's up to you to make sure that the output of each layer
    makes sense as the input to the next layer.
    """
    def __init__(self, layers: List[Layer]) -> None:
        self.layers = layers

    def forward(self, input):
        """Just forward the input through the layers in order."""
        for layer in self.layers:
            input = layer.forward(input)
        return input

    def backward(self, gradient):
        """Just backpropagate the gradient through the layers in reverse."""
        for layer in reversed(self.layers):
            gradient = layer.backward(gradient)
        return gradient

    def params(self) -> Iterable[Tensor]:
        """Just return the params from each layer."""
        return (param for layer in self.layers for param in layer.params())

    def grads(self) -> Iterable[Tensor]:
        """Just return the grads from each layer."""
        return (grad for layer in self.layers for grad in layer.grads())

#### So we could represent the neural network we used for XOR as:

In [ ]:
xor_net = Sequential([
    Linear(input_dim=2, output_dim=2),
    Sigmoid(),
    Linear(input_dim=2, output_dim=1),
    Sigmoid()
])

#### But we still need a little more machinery to train it.

# Loss and Optimization

#### Previously we wrote out individual loss functions and gradient functions for our models. Here we'll want to experiment with different loss functions, so (as usual) we'll introduce a new `Loss` abstractions that encapsulates both the loss computation and the gradient computation

In [ ]:
class Loss:
    def loss(self, predicted: Tensor, actual: Tensor) -> float:
        """How good are our predictions? (Larger numbers are worse.)"""
        raise NotImplementedError

    def gradient(self, predicted: Tensor, actual: Tensor) -> Tensor:
        """How does the loss change as the predictions change?"""
        raise NotImplementedError

class SSE(Loss):
    """Loss function that computes the sum of the squared errors."""
    def loss(self, predicted: Tensor, actual: Tensor) -> float:
        # Compute the tensor of squared differences
        squared_errors = tensor_combine(
            lambda predicted, actual: (predicted - actual) ** 2,
            predicted,
            actual)

        # And just add them up
        return tensor_sum(squared_errors)

    def gradient(self, predicted: Tensor, actual: Tensor) -> Tensor:
        return tensor_combine(
            lambda predicted, actual: 2 * (predicted - actual),
            predicted,
            actual)

#### The last piece to figure out is gradient descent. Throughout the repository we've done all of our gradient descent manually by having a training loop that involves something like:
` 
theta = gradient_step(theta, grad, -learning_rate)
`
#### Here that won't wwork for us, for a couple reasons. The first is that our neural nets will have many parameters, and we'll need to update all of them. The second is that we'd like to be able to use more clever varients of gradient descent, and we don't wwant to have to rewrite them each time. Accordingly, we'll introduce a (you guessed it) `Optimizer` abstraction, of which gradient descent will be a specific instance.

In [ ]:
class Optimizer:
    """ 
    An optimizer updates the weights of a layer (in place) using information 
    known by either the layer or the optimizer (or both).
    """
    def step(self, layer: Layer) -> None:
        raise NotImplementedError

In [ ]:

class GradientDescent(Optimizer):
    def __init__(self, learning_rate: float = 0.1) -> None:
        self.lr = learning_rate

    def step(self, layer: Layer) -> None:
        for param, grad in zip(layer.params(), layer.grads()):
            # Update param using a gradient step
            param[:] = tensor_combine(
                lambda param, grad: param - grad * self.lr,
                param,
                grad)

#### The only thing that's maybe surprising is the "slice assignment", which is a reflection of the fact that reassigning a list doesn't change its original value. That is, if you just did `param = tensor_combine(. . .)`, you would be redefining the local variable `param`, but you would not be affecting the original parameter tensor stored in the layer. If you assign to the slice [:], however, it actually changes the values inside the list.

#### To demonstrate the value of this abstraction, let's implement another optimizer that uses *momentum*. The idea is that we don't want to overreact to each new gradient, and so we maintain a running average of the gradients we've seen, updating it with each new gradient and taking a step in the direction of the average.

In [ ]:
class Momentum(Optimizer):
    def __init__(self,
                 learning_rate: float,
                 momentum: float = 0.9) -> None:
        self.lr = learning_rate
        self.mo = momentum
        self.updates: List[Tensor] = []  # running average

    def step(self, layer: Layer) -> None:
        # If we have no previous updates, start with all zeros.
        if not self.updates:
            self.updates = [zeros_like(grad) for grad in layer.grads()]

        for update, param, grad in zip(self.updates,
                                       layer.params(),
                                       layer.grads()):
            # Apply momentum
            update[:] = tensor_combine(
                lambda u, g: self.mo * u + (1 - self.mo) * g,
                update,
                grad)

            # Then take a gradient step
            param[:] = tensor_combine(
                lambda p, u: p - self.lr * u,
                param,
                update) 

#### Let's see how easy it is to use our new framework to train a network that can compute XOR. We start by re-creating the training data:

In [ ]:
# training data
xs = [[0., 0], [0., 1], [1., 0], [1., 1]]
ys = [[0.],[1.],[1.],[0.]]

#### and then we define the network, although now we can leave off the last sigmoid layer:

In [ ]:
random.seed(0)

net = Sequential([
    Linear(input_dim=2, output_dim=2),
    Sigmoid(),
    Linear(input_dim=2, output_dim=1)
])

#### We can now write a simple training loop, except that now we can use the abstractions od `Optimizer` and `Loss`. This allows us to easily try different ones:

In [ ]:

xs = [[0., 0], [0., 1], [1., 0], [1., 1]]
ys = [[0.], [1.], [1.], [0.]]

random.seed(0)

net = Sequential([
    Linear(input_dim=2, output_dim=2),
    Sigmoid(),
    Linear(input_dim=2, output_dim=1)
])

import tqdm

optimizer = GradientDescent(learning_rate=0.1)
loss = SSE()

with tqdm.trange(3000) as t:
    for epoch in t:
        epoch_loss = 0.0

        for x, y in zip(xs, ys):
            predicted = net.forward(x)
            epoch_loss += loss.loss(predicted, y)
            gradient = loss.gradient(predicted, y)
            net.backward(gradient)

            optimizer.step(net)

        t.set_description(f"xor loss {epoch_loss:.3f}")

for param in net.params():
    print(param)